In [1]:
import os
import re
import math
import string
import random
from glob import glob
from tqdm import tqdm
import numpy as np
from argparse import Namespace

import tensorflow as tf
print(tf.__version__)
from tensorflow.keras import layers
from tensorflow.keras import models

2023-04-16 05:53:28.436065: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-16 05:53:28.635139: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-16 05:53:29.565935: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-04-16 05:53:29.566100: W tensorflow/stream_executor

2.10.0


In [2]:
NUM_JOINTS = 543
IMG_H = 56
IMG_W = 56

LIP = [
    61, 185, 40, 39, 37, 0, 267, 269, 270, 409,
    291, 146, 91, 181, 84, 17, 314, 405, 321, 375,
    78, 191, 80, 81, 82, 13, 312, 311, 310, 415,
    95, 88, 178, 87, 14, 317, 402, 318, 324, 308,
]

RIGHT_EYE = [
    246, 161, 160, 159, 158, 157, 173,
    33, 7, 163, 144, 145, 153, 154, 155, 133,
    247, 30, 29, 27, 28, 56, 190,
    130, 25, 110, 24, 23, 22, 26, 112, 243,
    113, 225, 224, 223, 222, 221, 189,
    226, 31, 228, 229, 230, 231, 232, 233, 244,
    143, 111, 117, 118, 119, 120, 121, 128, 245,
]

LEFT_EYE = [
    466, 387, 386, 385, 384, 398,
    263, 249, 390, 373, 374, 380, 381, 382, 362,
    467, 260, 259, 257, 258, 286, 414,
    359, 255, 339, 254, 253, 252, 256, 341, 463,
    342, 445, 444, 443, 442, 441, 413,
    446, 261, 448, 449, 450, 451, 452, 453, 464,
    372, 340, 346, 347, 348, 349, 350, 357, 465,
]

LEFT_HAND = [
    468, 469, 470, 471, 472, 473, 474, 475,
    476, 477, 478, 479, 480, 481, 482, 483,
    484, 485, 486, 487, 488
]

RIGHT_HAND = [
    522, 523, 524, 525, 526, 527, 528, 529,
    530, 531, 532, 533, 534, 535, 536, 537,
    538, 539, 540, 541, 542
]

POSE = [
    489, 490, 491, 492, 493, 494, 495, 496, 497,
    498, 499, 500, 501, 502, 503, 504, 505, 506,
    507, 508, 509, 510, 511, 512, 513, 514, 515,
    516, 517, 518, 519, 520, 521
]

In [3]:
def natural_keys(text):
    ""
    def atoi(text):
        return int(text) if text.isdigit() else text
    
    return [atoi(c) for c in re.split(r'(\d+)', text)]

tfrecords = glob("../data/tfrecords/*.tfrec")
tfrecords = sorted(tfrecords, key=natural_keys)
print(tfrecords)

['../data/tfrecords/stratified_split_0.tfrec', '../data/tfrecords/stratified_split_1.tfrec', '../data/tfrecords/stratified_split_2.tfrec', '../data/tfrecords/stratified_split_3.tfrec', '../data/tfrecords/stratified_split_4.tfrec', '../data/tfrecords/stratified_split_5.tfrec', '../data/tfrecords/stratified_split_6.tfrec', '../data/tfrecords/stratified_split_7.tfrec', '../data/tfrecords/stratified_split_8.tfrec', '../data/tfrecords/stratified_split_9.tfrec', '../data/tfrecords/stratified_split_10.tfrec', '../data/tfrecords/stratified_split_11.tfrec', '../data/tfrecords/stratified_split_12.tfrec', '../data/tfrecords/stratified_split_13.tfrec', '../data/tfrecords/stratified_split_14.tfrec', '../data/tfrecords/stratified_split_15.tfrec', '../data/tfrecords/stratified_split_16.tfrec', '../data/tfrecords/stratified_split_17.tfrec', '../data/tfrecords/stratified_split_18.tfrec', '../data/tfrecords/stratified_split_19.tfrec', '../data/tfrecords/stratified_split_20.tfrec', '../data/tfrecords/str

In [134]:
def parse_sequence(serialized_sequence):
    return tf.io.parse_tensor(
        serialized_sequence,
        out_type=tf.float32,
    )


def parse_tfrecord_fn(example):
    feature_description = {
        "n_frames": tf.io.FixedLenFeature([], tf.float32),
        "frames": tf.io.FixedLenFeature([], tf.string),
        "label": tf.io.FixedLenFeature([], tf.int64),
    }
    
    return tf.io.parse_single_example(example, feature_description)


def parse_data(example):
    # Parse Frames
    n_frames = example["n_frames"]
    frames = tf.reshape(parse_sequence(example["frames"]), shape=(n_frames, 543, 3))

    # Parse Labels
    label = example["label"]

    return n_frames, frames, label

AUTOTUNE = tf.data.AUTOTUNE


ds = tf.data.TFRecordDataset(tfrecords)

dataloader = (
    ds
    .map(parse_tfrecord_fn, num_parallel_calls=AUTOTUNE)
    .map(parse_data, num_parallel_calls=AUTOTUNE)
    .prefetch(AUTOTUNE)
)

In [5]:
@tf.function
def generate_a_heatmap(arr, centers, max_values):
    sigma = 0.1
    img_h, img_w = arr.shape

    for center, max_value in zip(centers, max_values):
        mu_x, mu_y = center[0], center[1]
        if not (np.isnan(mu_x) and np.isnan(mu_y)):
            # scale
            mu_x = min(math.floor(mu_x * img_w), img_w - 1)
            mu_y = min(math.floor(mu_y * img_h), img_h - 1)

            st_x = max(int(mu_x - 0.5 * sigma), 0)
            ed_x = min(int(mu_x + 0.5 * sigma) + 1, img_w)
            st_y = max(int(mu_y - 0.5 * sigma), 0)
            ed_y = min(int(mu_y + 0.5 * sigma) + 1, img_h)
            x = np.arange(st_x, ed_x, 1, np.float32)
            y = np.arange(st_y, ed_y, 1, np.float32)

            # if the keypoint not in the heatmap coordinate system
            if not (len(x) and len(y)):
                continue
            y = y[:, None]

            patch = np.exp(-((x - mu_x)**2 + (y - mu_y)**2) / 2 / sigma**2)
            patch = patch * max_value
            arr[st_y:ed_y, st_x:ed_x] = np.maximum(arr[st_y:ed_y, st_x:ed_x], patch)

In [10]:
# @tf.function
# def get_3d_heatmap(ret, human_kps, num_frames):
    
#     for i, frame in enumerate(range(num_frames)):
#         arr = ret[i]
#         human = human_kps[i]

#         x, y = human[:,:1], human[:,1:2]

#         # TODO: Normalize the whole sequence together
#         x = (x-np.nanmin(x))/(np.nanmax(x)-np.nanmin(x))
#         y = (y-np.nanmin(y))/(np.nanmax(y)-np.nanmin(y))

#         human = np.squeeze(np.array(list(zip(x, y))), axis=-1)

#         kps = np.expand_dims(human, axis=0)
#         all_kpscores = np.ones((1,num_frames,NUM_JOINTS), dtype=np.float32)
#         kpscores = np.ones_like(all_kpscores[:, 0])

#         for i in range(107):
#             generate_a_heatmap(arr[i], kps[:, i], kpscores[:, i])
            
#     return ret

In [11]:
tf.keras.backend.clear_session()

In [129]:
def get_3d_heatmap(ret, humans, num_frames, sigma=0.2):
    x, y = humans[:,:,:1], humans[:,:,1:2]
    print(x, y)

    # MinMax Normalization
    x_min = tf.reduce_min(tf.where(tf.math.is_nan(x), tf.float32.max, x))
    x_max = tf.reduce_max(tf.where(tf.math.is_nan(x), tf.float32.min, x))
    y_min = tf.reduce_min(tf.where(tf.math.is_nan(y), tf.float32.max, y))
    y_max = tf.reduce_max(tf.where(tf.math.is_nan(y), tf.float32.min, y))

    x = (x - x_min) / (x_max - x_min)
    y = (y - y_min) / (y_max - y_min)

    # Scale
    x = tf.math.floor(x * IMG_W)
    y = tf.math.floor(y * IMG_H)

    # Get NaN mask
    kps = tf.concat([x, y], axis=-1)
    nan_mask = tf.math.is_nan(kps)

    st_x = tf.math.maximum(tf.cast(x - 0.5 * sigma, tf.int32), tf.constant(0, dtype=tf.int32))
    ed_x = tf.math.minimum(tf.cast(x + 0.5 * sigma, tf.int32) + 1, tf.cast(IMG_W, tf.int32))
    st_y = tf.math.maximum(tf.cast(y - 0.5 * sigma, tf.int32), tf.constant(0, dtype=tf.int32))
    ed_y = tf.math.minimum(tf.cast(y + 0.5 * sigma, tf.int32) + 1, tf.cast(IMG_H, tf.int32))

    i = tf.concat([st_x, ed_x-1], axis=-1)
    j = tf.concat([st_y, ed_y-1], axis=-1)[..., None]

    ix = (i - tf.cast(x, tf.int32))**2
    ix = tf.broadcast_to(tf.transpose(ix[..., tf.newaxis], (0,1,3,2)), (32, 107, 2, 2))
    jy = (j - tf.cast(y,tf.int32)[..., tf.newaxis])**2

    patch = tf.exp(-(ix+jy) / 2 / sigma**2)
    
    return patch, i, j

In [139]:
def pose2heatmap(example):
    num_frames, data, label = example
    
    faces = tf.gather(data, LIP, axis=1).numpy()
    poses = tf.gather(data, POSE, axis=1).numpy()[:,:-8]
    rhs = tf.gather(data, RIGHT_HAND, axis=1).numpy()
    lhs = tf.gather(data, LEFT_HAND, axis=1).numpy()

    humans = tf.concat([faces, poses, rhs, lhs], axis=1)
    
    # Sample frames - resize or pad/uniform sample. Doing simple resize.
    humans = tf.image.resize(humans, size=(32, 107), method="nearest")
    
    # Empty tensor for heatmap
    ret = tf.zeros((32, 107, IMG_H, IMG_W), dtype=tf.float32)

#     heatmap = get_3d_heatmap(ret, humans, num_frames)
    
#     return heatmap, label
    return ret, humans, num_frames

In [138]:
sample = next(iter(dataloader))
sample[1]

<tf.Tensor: shape=(23, 543, 3), dtype=float32, numpy=
array([[[ 0.49440014,  0.38046983, -0.03062646],
        [ 0.49601725,  0.3507348 , -0.05756483],
        [ 0.5008185 ,  0.35934305, -0.03028346],
        ...,
        [ 0.31373617,  0.41234398, -0.05269891],
        [ 0.35072815,  0.39958185, -0.06021732],
        [ 0.38579622,  0.4011007 , -0.06471767]],

       [[ 0.5011503 ,  0.38055426, -0.03156953],
        [ 0.49290648,  0.3493601 , -0.05817606],
        [ 0.49832708,  0.3581275 , -0.03118932],
        ...,
        [ 0.33588555,  0.3889878 , -0.0631762 ],
        [ 0.37742943,  0.3802519 , -0.07176355],
        [ 0.41639116,  0.38289747, -0.076529  ]],

       [[ 0.49847096,  0.37949273, -0.0309729 ],
        [ 0.49176967,  0.34847346, -0.05721497],
        [ 0.49767977,  0.3573629 , -0.03084622],
        ...,
        [ 0.35855788,  0.3844513 , -0.06097187],
        [ 0.4005313 ,  0.37769613, -0.07045607],
        [ 0.43907768,  0.37987   , -0.07860945]],

       ...,

      

In [146]:
ret, humans, _ = pose2heatmap(sample)
ret.shape, humans.shape

(TensorShape([32, 107, 56, 56]), TensorShape([32, 107, 3]))

In [147]:
humans

<tf.Tensor: shape=(32, 107, 3), dtype=float32, numpy=
array([[[ 4.6895200e-01,  3.9122039e-01,  1.3318895e-02],
        [ 4.6855748e-01,  3.8820282e-01,  3.4378599e-03],
        [ 4.7007626e-01,  3.8500535e-01, -6.9671771e-03],
        ...,
        [           nan,            nan,            nan],
        [           nan,            nan,            nan],
        [           nan,            nan,            nan]],

       [[ 4.6608999e-01,  3.9358661e-01,  1.3970846e-02],
        [ 4.6797937e-01,  3.9064983e-01,  3.9357482e-03],
        [ 4.7196886e-01,  3.8727656e-01, -6.8762675e-03],
        ...,
        [           nan,            nan,            nan],
        [           nan,            nan,            nan],
        [           nan,            nan,            nan]],

       [[ 4.6608999e-01,  3.9358661e-01,  1.3970846e-02],
        [ 4.6797937e-01,  3.9064983e-01,  3.9357482e-03],
        [ 4.7196886e-01,  3.8727656e-01, -6.8762675e-03],
        ...,
        [           nan,         

In [144]:
# patch, i, j = get_3d_heatmap(rett, humanst, num_framest)

In [149]:
x, y = humans[:,:,:1], humans[:,:,1:2]
x.shape, y.shape

(TensorShape([32, 107, 1]), TensorShape([32, 107, 1]))

In [150]:
# MinMax Normalization
x_min = tf.reduce_min(tf.where(tf.math.is_nan(x), tf.float32.max, x))
x_max = tf.reduce_max(tf.where(tf.math.is_nan(x), tf.float32.min, x))
y_min = tf.reduce_min(tf.where(tf.math.is_nan(y), tf.float32.max, y))
y_max = tf.reduce_max(tf.where(tf.math.is_nan(y), tf.float32.min, y))

x = (x - x_min) / (x_max - x_min)
y = (y - y_min) / (y_max - y_min)

In [153]:
x.shape, y.shape

(TensorShape([32, 107, 1]), TensorShape([32, 107, 1]))

In [154]:
# Scale
x = tf.math.floor(x * IMG_W)
y = tf.math.floor(y * IMG_H)

In [157]:
x.shape, y.shape

(TensorShape([32, 107, 1]), TensorShape([32, 107, 1]))

In [159]:
kps = tf.concat([x, y], axis=-1)
kps.shape

TensorShape([32, 107, 2])

In [161]:
sigma = 0.2
st_x = tf.math.maximum(tf.cast(x - 0.5 * sigma, tf.int32), tf.constant(0, dtype=tf.int32))
ed_x = tf.math.minimum(tf.cast(x + 0.5 * sigma, tf.int32) + 1, tf.cast(IMG_W, tf.int32))
st_y = tf.math.maximum(tf.cast(y - 0.5 * sigma, tf.int32), tf.constant(0, dtype=tf.int32))
ed_y = tf.math.minimum(tf.cast(y + 0.5 * sigma, tf.int32) + 1, tf.cast(IMG_H, tf.int32))

In [165]:
st_x[0][0], ed_x[0][0]

(<tf.Tensor: shape=(1,), dtype=int32, numpy=array([20], dtype=int32)>,
 <tf.Tensor: shape=(1,), dtype=int32, numpy=array([22], dtype=int32)>)

In [166]:
st_y[0][0], ed_y[0][0]

(<tf.Tensor: shape=(1,), dtype=int32, numpy=array([5], dtype=int32)>,
 <tf.Tensor: shape=(1,), dtype=int32, numpy=array([7], dtype=int32)>)

In [ ]:
i = tf.concat([st_x, ed_x-1], axis=-1)
j = tf.concat([st_y, ed_y-1], axis=-1)[..., None]

In [ ]:




ix = (i - tf.cast(x, tf.int32))**2
ix = tf.broadcast_to(tf.transpose(ix[..., tf.newaxis], (0,1,3,2)), (32, 107, 2, 2))
jy = (j - tf.cast(y,tf.int32)[..., tf.newaxis])**2

patch = tf.exp(-(ix+jy) / 2 / sigma**2)